In [1]:
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
import torch
print(torch.__version__)

2.1.0+cu118


In [3]:
!pip install ninja
#!conda install pytorch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1 cudatoolkit=11.3 -c pytorch
!pip install  dgl -f https://data.dgl.ai/wheels/cu121/repo.html
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels/cu121/repo.html
Looking in links: https://data.dgl.ai/wheels-test/repo.html


In [4]:
!pip install huggingface_hub omegaconf torch_redstone einops tqdm open3d

In [5]:
!pip install open_clip_torch h5py

In [6]:
!pip install --upgrade https://github.com/unlimblue/KNN_CUDA/releases/download/0.2/KNN_CUDA-0.2-py3-none-any.whl

In [7]:
!git clone https://github.com/GNNatan/SemNov_AML_DAAI_23-24.git

fatal: destination path 'SemNov_AML_DAAI_23-24' already exists and is not an empty directory.


In [8]:
%cd /content/SemNov_AML_DAAI_23-24

/content/SemNov_AML_DAAI_23-24


In [9]:
!pip install -r requirements.txt

In [10]:
!sh download_data.sh

============Downloading ModelNet40 + OOD Splits in 
--2024-06-10 17:02:13--  https://www.dropbox.com/s/c2x3h59nxprjs21/modelnet40_normal_resampled.tar?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/c2x3h59nxprjs21/modelnet40_normal_resampled.tar [following]
--2024-06-10 17:02:13--  https://www.dropbox.com/s/dl/c2x3h59nxprjs21/modelnet40_normal_resampled.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbbdec2e24c590c380a755f312c.dl.dropboxusercontent.com/cd/0/get/CUkrO5Smh_LXBbafO9OmEyG4O3Yr_ZN1lIWKt4vH1O5DYVzrQIR7wC5MQj7ejKUD390eocX-pNGz1-kmq0l32zqA05bBugMqGl7f1ZZ_QsTSZRCnrrz-v9TItSIbyrIcoZMV5bUIbfnt3O3D27WNbuSM/file?dl=1# [following]
--2024-06-10 17:02:14--  https://ucbbdec2e24c590c380a755f312c.dl.dropboxusercontent.com/

In [11]:
import numpy as np
import open3d as o3d
import random
import torch
import sys
from openshape.src.param import parse_args
import openshape.src.models as models
from openshape.src.utils.data import normalize_pc
from openshape.src.utils.misc import load_config
from huggingface_hub import hf_hub_download
from utils.ood_utils import *
from utils.utils import *
from datasets.modelnet import *
from datasets.scanobject import *
from collections import OrderedDict
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
import open_clip
import re
from PIL import Image
import torch.nn.functional as F
import torch.nn as nn
import h5py
import tqdm
from knn_cuda import KNN

Cannot import torchlars


In [100]:
def load_pcd(pcd, num_points=2048, y_up=True, normalize=True):
    #pcd = o3d.io.read_point_cloud(file_name)
    xyz = np.asarray(pcd.points)
    rgb = np.asarray(pcd.colors)
    if rgb is None:
      rgb = np.ones_like(xyz) * 0.4
    n = xyz.shape[0]
    if n > num_points:
        idx = random.sample(range(n), num_points)
        xyz = xyz[idx]
        rgb = rgb[idx]
    if y_up:
        # swap y and z axis
        xyz[:, [1, 2]] = xyz[:, [2, 1]]
    if normalize:
      xyz = normalize_pc(xyz)
    features = np.concatenate([xyz, rgb], axis=1)
    xyz = torch.from_numpy(xyz).type(torch.float32)
    features = torch.from_numpy(features).type(torch.float32)
    return torch.stack([xyz]).float(), torch.stack([features]).cuda()

In [13]:
def load_model(config, model_name="OpenShape/openshape-pointbert-vitg14-rgb"):
    model = models.make(config).cuda()

    model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)

    checkpoint = torch.load(hf_hub_download(repo_id=model_name, filename="model.pt"))
    model_dict = OrderedDict()
    pattern = re.compile('module.')
    for k,v in checkpoint['state_dict'].items():
        if re.search("module", k):
            model_dict[re.sub(pattern, '', k)] = v
    model.load_state_dict(model_dict)
    return model

In [64]:
@torch.no_grad()
def extract_text_feat(texts, clip_model,):
    text_tokens = open_clip.tokenizer.tokenize(texts).cuda()
    return clip_model.encode_text(text_tokens)

In [63]:
@torch.no_grad()
def extract_image_feat(images, clip_model, clip_preprocess):
    image_tensors = [clip_preprocess(image) for image in images]
    image_tensors = torch.stack(image_tensors, dim=0).float().cuda()
    image_features = clip_model.encode_image(image_tensors)
    image_features = image_features.reshape((-1, image_features.shape[-1]))
    return image_features

In [61]:
@torch.no_grad()
def text_scores(pcd, classes = ["chair", "shelf", "door", "sink", "sofa"], model=None, clip = None):
    cli_args, extras = parse_args(sys.argv[1:])
    config = load_config("openshape/src/configs/train.yaml", cli_args = vars(cli_args), extra_args = extras)
    if model is None:
      model = load_model(config)
      model.eval()

    if clip is None:
      clip, _, clip_preprocess = open_clip.create_model_and_transforms('ViT-bigG-14', pretrained='laion2b_s39b_b160k', cache_dir="/kaiming-fast-vol/workspace/open_clip_model/")
      clip.cuda().eval()

    xyz, feat = load_pcd(pcd)
    shape_feat = model(xyz, feat, device='cuda', quantization_size=config.model.voxel_size)
    text_feat = extract_text_feat(classes, clip)
    scores = F.normalize(shape_feat, dim=1) @ F.normalize(text_feat, dim=1).T
    return scores

In [60]:
@torch.no_grad()
def image_scores(pcd, images, model=None, clip=None):
    cli_args, extras = parse_args(sys.argv[1:])
    config = load_config("openshape/src/configs/train.yaml", cli_args = vars(cli_args), extra_args = extras)
    if model is None:
      model = load_model(config)
      model.eval()
    if clip is None:
      clip_model, _, clip_preprocess = open_clip.create_model_and_transforms('ViT-bigG-14', pretrained='laion2b_s39b_b160k', cache_dir="/kaiming-fast-vol/workspace/open_clip_model/")
      clip.cuda().eval()
    else:
      clip_model, clip_preprocess = clip

    xyz, feat = load_pcd(pcd)
    shape_feat = model(xyz, feat, device='cuda', quantization_size=config.model.voxel_size)
    image_feat = extract_image_feat(images, clip_model, clip_preprocess)
    scores = F.normalize(shape_feat, dim=1) @ F.normalize(image_feat, dim=1).T
    return scores

In [51]:
def load_data(filename = 'openshape/data/sonn.h5'):
  sonn = h5py.File(filename, 'r')
  data = sonn['data'][:]
  label = sonn['label'][:]
  sonn.close()
  return data, label

In [59]:
SR1_classes = ["chair", "shelf", "door", "sink", "sofa"]
SR2_classes = ["bed", "toilet", "desk", "display", "table"]

SR1_images = [Image.open(f).convert("RGB") for f in [f"openshape/images/{category}.png" for category in SR1_classes]]
SR2_images = [Image.open(f).convert("RGB") for f in [f"openshape/images/{category}.png" for category in SR2_classes]]

In [53]:
data, _ = load_data()

In [14]:
cli_args, extras = parse_args(sys.argv[1:])
config = load_config("openshape/src/configs/train.yaml", cli_args = vars(cli_args), extra_args = extras)

In [15]:
model = load_model(config)
model = model.cuda().eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.pt:   0%|          | 0.00/388M [00:00<?, ?B/s]

In [54]:
sr1_text_scores = list()
sr2_text_scores = list()
sr1_img_scores = list()
sr2_img_scores = list()

clip, _, clip_preprocess = open_clip.create_model_and_transforms('ViT-bigG-14', pretrained='laion2b_s39b_b160k', cache_dir="/kaiming-fast-vol/workspace/open_clip_model/")
clip.cuda().eval()

open_clip_pytorch_model.bin:   0%|          | 0.00/10.2G [00:00<?, ?B/s]

  0%|          | 0/2890 [00:00<?, ?it/s]


NameError: name 'text_scores' is not defined

In [ ]:
for cloud in tqdm.tqdm(data):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud)
    pcd.colors = o3d.utility.Vector3dVector(np.ones_like(cloud) * 0.4)
    sr1_text_score = text_scores(pcd, classes=SR1_classes, model=model, clip=clip)
    sr2_text_score = text_scores(pcd, classes=SR2_classes, model=model, clip=clip)
    sr1_text_scores.append(sr1_text_score)
    sr2_text_scores.append(sr2_text_score)
    sr1_img_score = image_scores(pcd, images=SR1_images, model=model, clip=(clip, clip_preprocess))
    sr2_img_score = image_scores(pcd, images=SR2_images, model=model, clip=(clip, clip_preprocess))
    sr1_img_scores.append(sr1_img_score)
    sr2_img_scores.append(sr2_img_score)

In [ ]:
torch.save(sr1_text_scores, 'outputs/tensors/OpenShape_SR1/txt_scores.pt')
torch.save(sr2_text_scores, 'outputs/tensors/OpenShape_SR2/txt_scores.pt')
torch.save(sr1_img_scores, 'outputs/tensors/OpenShape_SR1/img_scores.pt')
torch.save(sr2_img_scores, 'outputs/tensors/OpenShape_SR2/img_scores.pt')

In [95]:
def tensor_to_pcd(tensor):
    points = tensor.cpu().numpy().astype(np.float64)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(np.ones_like(points) * 0.4)

    return pcd

In [16]:
#OPTIONS
data_root = './3D_OS_release_data'
num_points = 1024
batch_size = 1
num_workers = 4
sonn_split = 'main_split'
sonn_h5_name = 'objectdataset.h5'
num_points_test = 2048
drop_last = False
ws, rank = get_ws(), get_rank()

In [98]:
@torch.no_grad()
def get_feats(model, loader):
    """ DDP impl """
    all_feats = []
    all_labels = []
    model.eval()
    for i, batch in tqdm.tqdm(enumerate(loader, 0)):
        points, labels = batch[0], batch[1]
        assert torch.is_tensor(points), "expected BNC tensor as batch[0]"
        points = points.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)
        xyz, features = load_pcd(tensor_to_pcd(points[0]), normalize=False)
        feats = model(xyz, features)
        if is_dist() and ws > 1:
            feats = gather(feats, dim=0)
            labels = gather(labels, dim=0)
        all_feats.append(feats)
        all_labels.append(labels)
    all_feats = torch.cat(all_feats)
    all_labels = torch.cat(all_labels, dim=0)
    return all_feats, all_labels

In [73]:
dataloader_config = {
        'batch_size': batch_size, 'drop_last': drop_last, 'shuffle': False,
        'num_workers': num_workers, 'sampler': None, 'worker_init_fn': init_np_seed}

sonn_args = {
        'data_root': data_root,
        'sonn_split': sonn_split,
        'h5_file': sonn_h5_name,
        'split': 'all',  # we use both training (unused) and test samples during evaluation
        'num_points': num_points_test,  # default: use all 2048 sonn points to avoid sampling randomicity
        'transforms': None  # no augmentation applied at inference time
    }

In [74]:
sr1_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet1", **sonn_args), **dataloader_config)
sr2_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet2", **sonn_args), **dataloader_config)
sr3_loader = DataLoader(ScanObject(class_choice="sonn_ood_common", **sonn_args), **dataloader_config)

ScanObject - num_points: 2048, sonn_split: main_split, h5_suffix: objectdataset.h5, split: all, class_choice: {4: 0, 8: 1, 7: 2, 12: 3, 13: 4}, num samples: 1255
ScanObject - num_points: 2048, sonn_split: main_split, h5_suffix: objectdataset.h5, split: all, class_choice: {10: 0, 14: 1, 5: 2, 6: 3, 9: 2}, num samples: 788
ScanObject - num_points: 2048, sonn_split: main_split, h5_suffix: objectdataset.h5, split: all, class_choice: {0: 404, 1: 404, 2: 404, 3: 404, 11: 404}, num samples: 847


In [101]:
for params in ((sr1_loader, sr2_loader, "SR1"), (sr2_loader, sr1_loader, "SR2")):
  src_loader, tar1_loader = params[0], params[1]
  tar2_loader = sr3_loader
  src_name = params[2]
  train_data = ModelNet40_OOD(data_root=data_root, train=True,  num_points=num_points, class_choice=src_name, transforms=None)
  test_data  = ModelNet40_OOD(data_root=data_root, train=False, num_points=num_points, class_choice=src_name, transforms=None)
  train_sampler = DistributedSampler(train_data, num_replicas=ws, rank=rank, shuffle=True)
  test_sampler = DistributedSampler(test_data, num_replicas=ws, rank=rank, shuffle=True)
  train_loader = DataLoader(train_data, batch_size=batch_size, drop_last=drop_last, num_workers=num_workers, sampler=train_sampler, worker_init_fn=init_np_seed)
  test_loader = DataLoader(test_data, batch_size=batch_size, drop_last=drop_last, num_workers=num_workers, sampler=test_sampler, worker_init_fn=init_np_seed)
  knn = KNN(k=1, transpose_mode=True)
  train_feats, train_labels = get_feats(model, train_loader)
  src_feats, src_labels = get_feats(model, src_loader)
  tar1_feats, tar1_labels = get_feats(model, tar1_loader)
  tar2_feats, tar2_labels = get_feats(model, tar2_loader)
  train_labels = train_labels.cpu().numpy()

      # eucl distance in a non-normalized space
  src_dist, src_ids = knn(train_feats.unsqueeze(0), src_feats.unsqueeze(0))
  src_dist = src_dist.squeeze().cpu()
  src_ids = src_ids.squeeze().cpu()  # index of nearest training sample

      # OOD tar1
  tar1_dist, tar1_ids = knn(train_feats.unsqueeze(0), tar1_feats.unsqueeze(0))
  tar1_dist = tar1_dist.squeeze().cpu()
  tar1_ids = tar1_ids.squeeze().cpu()
      # OOD tar2
  tar2_dist, tar2_ids = knn(train_feats.unsqueeze(0), tar2_feats.unsqueeze(0))
  tar2_dist = tar2_dist.squeeze().cpu()
  tar2_ids = tar2_ids.squeeze().cpu()

    #save
  torch.save(src_dist, f'outputs/tensors/OpenShape_{src_name}/src_dist.pt')
  torch.save(src_ids, f'outputs/tensors/OpenShape_{src_name}/src_ids.pt')
  torch.save(tar1_dist, f'outputs/tensors/OpenShape_{src_name}/tar1_dist.pt')
  torch.save(tar1_ids, f'outputs/tensors/OpenShape_{src_name}/tar1_ids.pt')
  torch.save(tar2_dist, f'outputs/tensors/OpenShape_{src_name}/tar2_dist.pt')
  torch.save(tar2_ids, f'outputs/tensors/OpenShape_{src_name}/tar2_ids.pt')

ModelNet40_OOD - Reading data from h5py file: ./3D_OS_release_data/modelnet40_normal_resampled/ood_sets_cache/SR1_train.h5
ModelNet40_OOD - split: train, categories: {'chair': 0, 'bookshelf': 1, 'door': 2, 'sink': 3, 'sofa': 4}
ModelNet40_OOD - Reading data from h5py file: ./3D_OS_release_data/modelnet40_normal_resampled/ood_sets_cache/SR1_test.h5
ModelNet40_OOD - split: test, categories: {'chair': 0, 'bookshelf': 1, 'door': 2, 'sink': 3, 'sofa': 4}


2378it [00:54, 43.67it/s]
1255it [00:38, 32.88it/s]
788it [00:24, 32.22it/s]
847it [00:25, 32.79it/s]


ModelNet40_OOD - Reading data from h5py file: ./3D_OS_release_data/modelnet40_normal_resampled/ood_sets_cache/SR2_train.h5
ModelNet40_OOD - split: train, categories: {'bed': 0, 'toilet': 1, 'desk': 2, 'monitor': 3, 'table': 2}
ModelNet40_OOD - Reading data from h5py file: ./3D_OS_release_data/modelnet40_normal_resampled/ood_sets_cache/SR2_test.h5
ModelNet40_OOD - split: test, categories: {'bed': 0, 'toilet': 1, 'desk': 2, 'monitor': 3, 'table': 2}


1916it [00:43, 44.15it/s]
788it [00:24, 31.53it/s]
1255it [00:38, 32.84it/s]
847it [00:25, 32.62it/s]
